In [ ]:
!pip install pdfplumber pymupdf pandas openpyxl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 78.8 MB/s eta 0:00:00


In [ ]:
import os
import fitz  # PyMuPDF
import pandas as pd
import re
from google.colab import files

def clean_text(text):
    """
    Removes illegal characters that cannot be stored in Excel.
    """
    text = re.sub(r'[^\x20-\x7E]', '', text)  # Remove non-printable ASCII
    return text.strip()

def extract_tables_from_pdf(pdf_path, output_excel):
    """
    Extracts tables from PDFs, removes illegal characters, and saves them as an Excel file.
    """
    print(f"Processing PDF: {pdf_path}")
    doc = fitz.open(pdf_path)
    all_tables = []

    for i, page in enumerate(doc):
        text = page.get_text("text")  # Extract text
        lines = text.split("\n")
        table_data = [[clean_text(cell) for cell in line.split()] for line in lines if line.strip()]

        if len(table_data) > 1:  # Ensure there's enough data to form a table
            df = pd.DataFrame(table_data)
            df_cleaned = clean_dataframe(df)
            if not df_cleaned.empty:
                all_tables.append((i+1, df_cleaned))

    if all_tables:
        with pd.ExcelWriter(output_excel) as writer:
            for page, df in all_tables:
                df.to_excel(writer, sheet_name=f'Page_{page}', index=False, header=False)

        print(f"✅ Tables extracted and saved in {output_excel}")
    else:
        print(f"❌ No tables detected in {pdf_path}")

def clean_dataframe(df):
    """
    Cleans extracted tables by:
    - Removing empty rows/columns
    - Standardizing formatting
    - Ensuring correct row alignment
    - Removing non-printable characters
    """
    df = df.dropna(how="all", axis=0)  # Remove empty rows
    df = df.dropna(how="all", axis=1)  # Remove empty columns
    df = df.applymap(lambda x: clean_text(x) if isinstance(x, str) else x)  # Remove illegal characters
    df.fillna("", inplace=True)  # Fill missing values
    return df

def process_multiple_pdfs():
    """
    Uploads and processes multiple PDFs, saving each as a separate Excel file.
    """
    uploaded_files = files.upload()  # Upload multiple PDFs
    for pdf_file in uploaded_files.keys():
        output_excel = f"{os.path.splitext(pdf_file)[0]}.xlsx"
        extract_tables_from_pdf(pdf_file, output_excel)

    print("✅ All PDFs processed successfully!")

# Run this to upload and process multiple PDFs
process_multiple_pdfs()




Saving test3 (1) (1).pdf to test3 (1) (1) (3).pdf
Saving test6 (1) (1).pdf to test6 (1) (1) (2).pdf
Processing PDF: test3 (1) (1) (3).pdf
✅ Tables extracted and saved in test3 (1) (1) (3).xlsx
Processing PDF: test6 (1) (1) (2).pdf


<ipython-input-10-3c0a2e501044>:52: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: clean_text(x) if isinstance(x, str) else x)  # Remove illegal characters
<ipython-input-10-3c0a2e501044>:52: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: clean_text(x) if isinstance(x, str) else x)  # Remove illegal characters
<ipython-input-10-3c0a2e501044>:52: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: clean_text(x) if isinstance(x, str) else x)  # Remove illegal characters
<ipython-input-10-3c0a2e501044>:52: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: clean_text(x) if isinstance(x, str) else x)  # Remove illegal characters
<ipython-input-10-3c0a2e501044>:52: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  

✅ Tables extracted and saved in test6 (1) (1) (2).xlsx
✅ All PDFs processed successfully!


In [ ]:
from google.colab import files
files.download("extracted_tables.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>